## Métrica desconocimiento

In [1]:
from dbconn import querydb, querydbtopandas 
import pandas as pd 

In [2]:
mayor = "2019-12"
menor = "2019-10"

In [15]:
df = querydbtopandas(f"""select    a11.id_encuesta  id_encuesta,
                
                a13.id_pre_tipo,


                a11.id_afiliado  id_afi_afiliado,

                a11.id_prestador  id_pre_prestador,

                max(a13.desc_pre_nombre)  desc_pre_nombre,

                max(a13.vigente)  vigente,

                a11.id_transac_tipo  id_transac_tipo,

                a11.id_prest_demandante  id_prest_demandante,

                max(a16.desc_pre_nombre)  desc_pre_nombre_demandante,

                a11.id_prest_prestacion  id_pres_prestacion,

                max(a18.desc_pres_prestacion)  desc_pres_prestacion,

                max(a18.prestac_pres_prestacion)  prestac_pres_prestacion,

                sum(distinct a11.i_CNT_desc)  QDESCONOCIDAS

from      DBA.ft_prest_encuestas              a11

                join        DBA.d_pre_prestador   a13

                  on        (a11.id_prestador = a13.id_pre_prestador)

                join        DBA.d_pre_prestador_demandante  a16

                  on        (a11.id_prest_demandante = a16.id_pre_prestador)

                join        DBA.d_pres_prestacion a18

                  on        (a11.id_prest_prestacion = a18.id_pres_prestacion)


where (cast(a11.fec_encuesta as date) >= date("{menor}") and cast(a11.fec_encuesta as date) < date("{mayor}")

and a11.id_moti_desconoc <> 1)

group by             a11.id_encuesta,

                    a13.id_pre_tipo,

                a11.id_afiliado,

                a11.id_prestador,
                
                a11.id_prest_demandante,
                
                 a16.id_pre_prestador, 

                a13.id_pre_especialidad,

                a11.id_transac_tipo,

                a11.id_prest_efector,

                a11.id_prest_prestacion,

                DATE(a11.fec_consumo),

                a11.id_tie_mes_c""")

In [16]:
clinicas_propias_id = [1632,1763,2086,26564,55060 ,64677,75470,78363 ,85316,85401 ,99785 ,118209 ,122803,135464 ,182730,187243 ,196559 ,196561 ,206939 ,210252 ,211421, 213729 ,217747 ,226756,235779,240895] 
cma = df[df["id_pre_prestador"].isin(clinicas_propias_id)][["id_pre_prestador", "desc_pre_nombre"]].rename(columns={"desc_pre_nombre": "cma"})
df = pd.merge(df, cma, how="outer").fillna("No aplica")

In [17]:
df

,id_encuesta,id_pre_tipo,id_afi_afiliado,id_pre_prestador,desc_pre_nombre,vigente,id_transac_tipo,id_prest_demandante,desc_pre_nombre_demandante,id_pres_prestacion,desc_pres_prestacion,prestac_pres_prestacion,QDESCONOCIDAS,cma
0,563532,C,3352792,65530,Colegio de Kinesiólogos de Santa Fe - 2° Circu...,A,0,182636,"Fuks,Mirko",1972,Módulo Fisio-Kinesio-Laser-Magnetoterapia,25010212,1.0,No aplica
1,569472,C,1967678,65530,Colegio de Kinesiólogos de Santa Fe - 2° Circu...,A,0,204659,"Del Prato,Martín J.",1972,Módulo Fisio-Kinesio-Laser-Magnetoterapia,25010212,1.0,No aplica
2,563533,C,3527735,65530,Colegio de Kinesiólogos de Santa Fe - 2° Circu...,A,0,182636,"Fuks,Mirko",1972,Módulo Fisio-Kinesio-Laser-Magnetoterapia,25010212,1.0,No aplica
3,569477,C,1967678,65530,Colegio de Kinesiólogos de Santa Fe - 2° Circu...,A,0,204659,"Del Prato,Martín J.",1972,Módulo Fisio-Kinesio-Laser-Magnetoterapia,25010212,1.0,No aplica
4,563534,C,3527735,65530,Colegio de Kinesiólogos de Santa Fe - 2° Circu...,A,0,182636,"Fuks,Mirko",1972,Módulo Fisio-Kinesio-Laser-Magnetoterapia,25010212,1.0,No aplica
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4896,537974,P,594944,20523,"Grau,Mirta L.",A,0,0,Sin Asignar,22448,"Psicoterapias individuales, niños o adultos, p...",33010100,1.0,No aplica
4897,582795,P,402160,153091,"San Millán,Ricardo J.",A,0,0,Sin Asignar,2084,Consulta Traumatología,42011200,1.0,No aplica
4898,566528,P,443597,157875,"Teijeiro,María Jimena",A,0,0,Sin Asignar,2096,Consulta Nutrición,42011901,1.0,No aplica
4899,563391,I,2842095,127022,Rehabplus - Lic. Gisela Warszawski,A,0,0,Sin Asignar,1972,Módulo Fisio-Kinesio-Laser-Magnetoterapia,25010212,1.0,No aplica


In [19]:
propios = df.cma.value_counts().index[1:]

In [20]:
df_part_equipos = df[df["id_pre_tipo"].isin(["P","E"])][["id_encuesta", "id_afi_afiliado", "id_pre_prestador", "desc_pre_nombre", "cma" ]]
df_inst_circ = df[df["id_pre_tipo"].isin(["I","C"])][["id_encuesta","id_afi_afiliado", "id_prest_demandante", "desc_pre_nombre_demandante","cma"]]
df_inst_circ.rename(columns={"id_prest_demandante": "id_pre_prestador", "desc_pre_nombre_demandante":"desc_pre_nombre"},inplace=True)

In [21]:
df = pd.concat([df_part_equipos, df_inst_circ])

In [22]:
df = df[df["id_pre_prestador"] != 0]

In [23]:
#count de encuestas
encuestas_desconocidas = df.groupby(["id_pre_prestador","desc_pre_nombre","cma"])["id_encuesta"].nunique()

In [24]:
#count de afiliados
afiliados_desconocidos = df.groupby(["id_pre_prestador","desc_pre_nombre", "cma"])["id_afi_afiliado"].nunique()

In [25]:
df = pd.concat([encuestas_desconocidas, afiliados_desconocidos],axis=1).reset_index()

In [26]:
df.rename(columns = {"id_pre_prestador":"Prestador",
                     "desc_pre_nombre":"Razón Social",
                     "id_encuesta":"Q desconocidas",
                     "id_afi_afiliado":"Q afiliados"},inplace=True)

In [27]:
df.sort_values(by= "Q afiliados", ascending = False).head(100)

,Prestador,Razón Social,cma,Q desconocidas,Q afiliados
14,2085,Clínica San Camilo,No aplica,42,25
271,105605,"Galich,Miriam",No aplica,15,13
445,182636,"Fuks,Mirko",No aplica,41,7
538,249069,"Salcedo,Anabel",SWISS MEDICAL CENTER MICROCENTRO SEDE 25 DE MAYO,8,7
160,67929,"Mosto,Julián",No aplica,5,5
...,...,...,...,...,...
52,22836,"Waxman,Viviana S.",No aplica,3,2
281,107586,"Bruno,Santiago J.",No aplica,4,2
28,20711,"Velázquez,Humberto",No aplica,2,2
326,123676,"García Bournissen,Facundo",No aplica,3,2


In [28]:
df

,Prestador,Razón Social,cma,Q desconocidas,Q afiliados
0,105,"Cohen,Diego E.",No aplica,10,3
1,172,"Pasqualini,Rodolfo S.",No aplica,2,2
2,232,"Fioroni,Andrea P.",No aplica,1,1
3,292,"Tejss de Acri,Cristina",No aplica,1,1
4,334,"Barclay,Fernando E.",No aplica,1,1
...,...,...,...,...,...
537,249069,"Salcedo,Anabel",SWISS MEDICAL CENTER BERAZATEGUI,1,1
538,249069,"Salcedo,Anabel",SWISS MEDICAL CENTER MICROCENTRO SEDE 25 DE MAYO,8,7
539,249069,"Salcedo,Anabel",SWISS MEDICAL CENTER MICROCENTRO SEDE SARMIENTO,3,3
540,249069,"Salcedo,Anabel",SWISS MEDICAL CENTER OLIVOS,2,2
